In [7]:
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from meteostat import Point, Daily, Hourly
import numpy as np
import pandas as pd

In [8]:
data = pd.read_csv("../processed/processed-shutoffs.csv")
data.head()

,circuit_name,deenergize_time,restoration_time,time_out_min,key_communities,hftd_tier,total_affected,residential_affected,zip_code,longitude,latitude,substn_present,total_population
0,APPLE HILL-1103,2018-10-14 21:00:00,2018-10-15 16:29:00.000000000,1169.0,"Camino, Placerville",3.0,1809.0,1488.0,95709,-120.677165,38.739683,True,5634.0
1,APPLE HILL-1104,2018-10-14 21:17:00,2018-10-15 18:37:00.000000000,1280.0,"Camino, Placerville, Pollock Pines,",3.0,2261.0,2043.0,95709,-120.677165,38.739683,True,5634.0
2,APPLE HILL-2102,2018-10-14 21:05:00,2018-10-16 18:04:00.000000000,2699.0,"Camino, Grizzly Flats, Mount Aukum, Placervill...",3.0,4489.0,4013.0,95709,-120.677165,38.739683,True,5634.0
3,BONNIE NOOK-1102,2018-10-14 21:54:00,2018-10-15 18:04:00.000000000,1210.0,Alta,3.0,533.0,454.0,95701,-120.809292,39.214708,True,1064.0
4,CALISTOGA-1101,2018-10-14 20:34:00,2018-10-15 21:27:00.000000000,1493.0,Calistoga,3.0,1596.0,1216.0,94515,-122.578610,38.592088,True,7525.0


In [21]:
data["deenergize_time"] = pd.to_datetime(data["deenergize_time"])
data["restoration_time"] = pd.to_datetime(data["restoration_time"])
data["deenergize_date"] = data["deenergize_time"].dt.date
data["restoration_date"] = data["restoration_time"].dt.date


In [78]:
pd.to_datetime(data.loc[i, "deenergize_date"])

Timestamp('2018-10-14 00:00:00')

In [115]:
missing = {}
full_data = {}
weather_present = {}
tmin_missing = {}
tmax_missing = {}
wspd_missing = {}
daily = {}
for i in range(data.shape[0]):
    point_1 = Point(data.loc[i, "latitude"], data.loc[i, "longitude"])
    start = pd.to_datetime(data.loc[i, "deenergize_date"]) - timedelta(days=5)
    end = pd.to_datetime(data.loc[i, "deenergize_date"])
    # weather = Hourly(point_1, start, end)
    weather = Daily(point_1, start, end)
    weather = weather.fetch()
    # if weather.shape[0] == 0:
    #     weather = Daily(point_1, start, end)
    #     weather = weather.fetch()
    if weather.shape[0] == 0:
        missing[i] = data.loc[i, ["circuit_name", "deenergize_time", "latitude", "longitude"]]
    #     daily[i] = weather
    elif weather.shape[0] >= 5:
        if weather.shape[0] > 5:
            weather = weather.loc[start.date(): end.date() - timedelta(days=1), :]
            assert weather.shape[0] == 5
        if sum(weather["tmin"].isna()) > 0:
            tmin_missing[i] = data.loc[i, ["circuit_name",
                                "deenergize_time", "latitude", "longitude"]]
        if sum(weather["tmax"].isna()) > 0:
            tmax_missing[i] = data.loc[i, ["circuit_name",
                               "deenergize_time", "latitude", "longitude"]]
        if sum(weather["wspd"].isna()) > 0:
            wspd_missing[i] = data.loc[i, ["circuit_name", "deenergize_time", "latitude", "longitude"]]
        if sum(np.sum(weather[["tmin", "tmax", "wspd"]].isna(), axis=1)) == 0:
            full_data[i] = data.loc[i, ["circuit_name", "deenergize_time", "latitude", "longitude"]]
            daily[i] = weather
    else:
        weather_present[i] = data.loc[i, ["circuit_name", "deenergize_time", "latitude", "longitude"]]
        

In [116]:
missing = pd.DataFrame(missing)
missing.T.to_csv("missing_weather.csv")



In [120]:
full_data = pd.DataFrame(full_data)
full_data.T.to_csv("full_weather.csv")

In [121]:
weather_present = pd.DataFrame(weather_present)
weather_present.T.to_csv("some_missing_weather.csv")
tmin_missing = pd.DataFrame(tmin_missing)
tmin_missing.T.to_csv("tmin_missing.csv")
tmax_missing = pd.DataFrame(tmax_missing)
tmax_missing.T.to_csv("tmax_missing.csv")
wspd_missing = pd.DataFrame(wspd_missing)
wspd_missing.T.to_csv("wspd_missing.csv")


In [123]:
for i in list(daily.keys()):
    shutoff = data.iloc[i, :]
    filename = f"raw/daily/weather_{shutoff['circuit_name']}_{shutoff['deenergize_date']}.csv"
    weather = daily[i]
    weather.to_csv(filename)
    

In [51]:
# for i in list(hourly.keys()):
#     shutoff = data.iloc[i, :]
#     filename = f"raw/hourly/weather_{shutoff['circuit_name']}_{shutoff['deenergize_date']}_hourly.csv"
#     weather = hourly[i]
#     weather.to_csv(filename)
